In [132]:
import pandas as pd

In [133]:
# create a list of all the texts
texts = []
with open('subset.txt', 'r') as f:
    for line in f:
        elements = line.strip().split('\t')
        texts.append(elements[0].replace('\n', ''))
        texts.append(elements[1])
        
for text in texts:
    print(text)

Turn off the main breaker.
Hot or not? Now turn the main on, and B2 off.
I think it would.
I don't think you should.
You don't need any visa.
You don't need sauce at all.
You don't have to know.
"I don't have to tell you."""
And if you're holding in a Roth IRA, you get that tax free.
Consider the tax bracket that you would be in if you rolled your traditional IRA into a Roth IRA.
In the first case, I think you don't need it.
If, however, you don't think that you can do the job - don't apply.
Can you do this?
Some of what you can do:
So I don't think you need to put it on the cover.
Look - I understand you need to do this quick and you know its wrong to put on vinyl.
It depends on how it's used.
It depends on your country.
I think there isn't a general answer.
I don't think that there's any.


In [134]:
# create list of shingles
shingles_per_text = []
k = 3
for text in texts:
    shingles_per_text.append(set([text[i:i + k] for i in range(len(text) - k + 1)]))
    
for s in shingles_per_text:
    print(s)

{' ma', 'rn ', 'bre', ' br', 'he ', 'Tur', 'in ', 'ake', 'n b', 'er.', 'eak', 'urn', 'e m', 'ker', 'f t', 'the', 'off', 'ff ', 'ain', 'rea', ' th', ' of', 'n o', 'mai'}
{' ma', 'rn ', ' or', ' B2', '2 o', 'he ', 'in ', 'Now', 'n t', 'ot ', 'nd ', 'urn', 'e m', 'w t', 'the', 'off', ' an', 'ain', 'ow ', ' no', 'ff.', ' th', 'on,', 'tur', ' on', 'not', 'n o', ' of', 'B2 ', 't? ', 'ot?', 'mai', ' tu', 'Hot', '? N', 'n, ', ', a', 'or ', 't o', 'r n', 'and', ' No', 'd B'}
{'nk ', 'hin', ' it', 'k i', 'thi', ' th', 'it ', ' wo', 'uld', 't w', 'wou', 'ld.', 'oul', 'ink', 'I t'}
{'you', "on'", 'I d', ' sh', ' do', 'sho', ' yo', "n't", 'ou ', 'k y', 'u s', "'t ", 'nk ', 'hin', 'thi', ' th', 'ld.', 'oul', 't t', 'hou', 'don', 'uld', 'ink'}
{'y v', 'isa', "on'", 'eed', ' do', 'd a', "n't", 'ou ', 'You', 'u d', 'ny ', "'t ", ' an', 'ed ', 'vis', 't n', 'nee', ' ne', 'any', 'don', 'sa.', ' vi'}
{'all', "on'", 'eed', ' do', 'd s', 'ce ', "n't", 'ou ', ' at', 'You', 'sau', 'at ', 'u d', "'t ", 'ed ', 

In [146]:
bit_matrix = []
for shingle_set in shingles_per_text:
    vector = dict()
    for shingle in shingle_set:
        hash_value = hash(shingle) % 5000
        vector[hash_value] = 1
    bit_matrix.append(vector)
print(bit_matrix)

[{4625: 1, 4586: 1, 1522: 1, 3733: 1, 911: 1, 4856: 1, 3177: 1, 3088: 1, 3744: 1, 2509: 1, 3753: 1, 2675: 1, 4708: 1, 3062: 1, 4575: 1, 2754: 1, 131: 1, 2038: 1, 1042: 1, 4377: 1, 3938: 1, 1826: 1, 683: 1, 784: 1}, {4625: 1, 4586: 1, 3227: 1, 2434: 1, 3257: 1, 911: 1, 3177: 1, 723: 1, 1339: 1, 155: 1, 2308: 1, 2675: 1, 4708: 1, 4584: 1, 2754: 1, 131: 1, 3469: 1, 1042: 1, 2700: 1, 3501: 1, 3104: 1, 3938: 1, 2306: 1, 4603: 1, 3235: 1, 4610: 1, 683: 1, 1826: 1, 718: 1, 4939: 1, 2368: 1, 784: 1, 1371: 1, 1572: 1, 2396: 1, 3814: 1, 3623: 1, 1920: 1, 4323: 1, 3500: 1, 1956: 1, 2680: 1, 508: 1}, {3442: 1, 3188: 1, 4668: 1, 4208: 1, 73: 1, 3938: 1, 1098: 1, 4281: 1, 3042: 1, 1119: 1, 261: 1, 950: 1, 4279: 1, 1305: 1, 1844: 1}, {4588: 1, 3849: 1, 1468: 1, 4925: 1, 3957: 1, 2775: 1, 1088: 1, 302: 1, 4444: 1, 133: 1, 2917: 1, 486: 1, 3442: 1, 3188: 1, 73: 1, 3938: 1, 950: 1, 4279: 1, 267: 1, 147: 1, 3056: 1, 3042: 1, 1305: 1}, {4753: 1, 4666: 1, 3849: 1, 4747: 1, 3957: 1, 2244: 1, 302: 1, 4444: 1

# calculate minhash signatures

In [136]:
import numpy as np
n = 50
hash_functions = []
for h in range(n):
    perm = np.random.permutation(5000)
    hash_functions.append(perm)

In [150]:
minhash_signatures = []
for vector in bit_matrix:
    signature = []
    for perm in hash_functions:
        for index in perm:
            if vector.get(index) == 1:
                signature.append(index)
                break
                
    minhash_signatures.append(signature)

In [151]:
for sig in minhash_signatures:
    print(sig)

[3177, 4575, 911, 683, 3938, 4377, 3088, 1826, 683, 1826, 911, 4586, 131, 4586, 4575, 3088, 4575, 3753, 911, 4856, 3733, 3733, 2754, 131, 2675, 2038, 4625, 3733, 1522, 2038, 1826, 3062, 2675, 784, 3177, 4856, 4586, 683, 3733, 2038, 4856, 4575, 911, 4377, 3753, 131, 683, 3177, 1042, 4575]
[2700, 2368, 1572, 683, 155, 3235, 3469, 1826, 1920, 1826, 911, 3104, 1371, 2434, 3501, 911, 4939, 3469, 4610, 4603, 3235, 3814, 4584, 131, 2675, 2700, 4625, 4708, 4584, 2754, 1826, 1920, 2675, 784, 3177, 2434, 3104, 683, 2434, 1042, 2680, 3501, 911, 508, 3500, 131, 683, 1371, 723, 1956]
[3042, 4281, 1119, 3188, 3938, 73, 4281, 4281, 4281, 3938, 1844, 4208, 3442, 4668, 1098, 4208, 4668, 4279, 4279, 3042, 950, 950, 1305, 3442, 3042, 3442, 3042, 1305, 1844, 1305, 4208, 3188, 261, 4279, 3188, 1844, 3188, 1305, 4281, 73, 1844, 4668, 1119, 3938, 1098, 3442, 1098, 1119, 4279, 950]
[267, 3056, 267, 3188, 3938, 2775, 267, 3849, 267, 133, 3938, 3056, 2917, 1468, 302, 1468, 3442, 3957, 4279, 302, 1088, 2917, 133

# LSH 

In [157]:
def sort_bands_into_buckets(number_of_buckets=50000, number_bands=16, number_rows_per_band=len(hash_functions) / 16):
    buckets = dict()
    for col_index in range(len(minhash_signatures)):
        for band_num in range(number_bands):
            # get the signature vector for the current text
            signature_vector = minhash_signatures[col_index]
            # get the current band
            start_index = int(band_num * number_rows_per_band)
            end_index = int((band_num + 1) * number_rows_per_band)
            band = signature_vector[start_index:end_index]
            # calculate the hash value for the band
            bucket_index = hash(str(band)) % number_of_buckets
            # add the text to the bucket
            buckets[bucket_index] = buckets.get(bucket_index, []) + [col_index]
            
    return buckets
       

In [158]:
buckets = sort_bands_into_buckets()

In [159]:
def find_candidate_pairs(buckets):    
    candidate_pairs = set()
    for bucket in buckets.values():
        for i in range(len(bucket)):
            for j in range(i + 1, len(bucket)):
                candidate_pairs.add((bucket[i], bucket[j]))
    return candidate_pairs

In [160]:
candidate_pairs = find_candidate_pairs(buckets)

In [161]:
print(len(candidate_pairs))

5


0
0
